####Querying Source

In [0]:
%sql
SELECT * FROM spark_cata.source.ab_products;

In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import *

df = spark.sql("SELECT * FROM spark_cata.source.ab_products") #Create a dataframe on the table
df = df.withColumn('row_num', row_number().over(Window.partitionBy('id').orderBy(desc('updated_date'))))

df = df.filter(df.row_num == 1).drop('row_num') #De Duplication


#####UPSERTS (Merge)

In [0]:
from delta.tables import *

if len(dbutils.fs.ls("/Volumes/spark_cata/source/db_volume/products_dst/")) > 0:
    delta_obj = DeltaTable.forPath(spark, "/Volumes/spark_cata/source/db_volume/products_dst/") #Create a delta object for thr catalog

    delta_obj.alias('trgt').merge(
        df.alias('src'),
        "src.id = trgt.id")\
        .whenMatchedUpdateAll(condition = "src.updated_date >= trgt.updated_date")\
        .whenNotMatchedInsertAll()\
        .execute()

else:

    df.write.format("delta")\
            .mode("Overwrite")\
            .save("/Volumes/spark_cata/source/db_volume/products_dst/")

In [0]:
%sql
SELECT * FROM delta.`/Volumes/spark_cata/source/db_volume/products_dst/`;